<a href="https://colab.research.google.com/github/d0rtilla/AbaloneAgePredictionProject/blob/main/Abalone_Age_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Extract and Transform Data

In [ ]:
# Import dependencies

In [ ]:
# Read in data

In [4]:
# Add column names

In [5]:
# Get dummies on sex column

### Multiple Linear Regression Model

In [6]:
# Create model

In [7]:
# Split into training and testing sets

In [9]:
# Scale data

In [10]:
# Train data

In [11]:
# Test model

In [12]:
# Evaluate model